In [1]:
import cv2

In [2]:
import os
import numpy as np

In [4]:
import matplotlib as plt
%matplotlib notebook

In [12]:
from IPython.html import widgets
from IPython.display import display

### Select working folder

In [5]:
%gui qt

In [6]:
try:
    from PySide import QtCore, QtGui
except ImportError:
    from PyQt4 import QtCore, QtGui

def gui_fname(dir=None, message=None):
    """Select a directory
    """
    if dir is None: dir ='../'
    if message is None: message = 'Select Folder ...'
    dir_name = QtGui.QFileDialog.getExistingDirectory(None, message,
                                                      dir,
                                                      QtGui.QFileDialog.ShowDirsOnly)
    return dir_name

In [7]:
%gui qt
data_folder = gui_fname()
%matplotlib inline

In [8]:
print("You have selected the following folder: ", data_folder)

You have selected the following folder:  /Users/j35/Desktop/Registration_sample/Cropped_RawData


In [9]:
list_of_files = [name for name in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, name))]
number_of_files = len(list_of_files)
print('Folder contains %d files.' %number_of_files)

Folder contains 959 files.


### Load the data 

In [28]:
w = widgets.IntProgress()
w.max = (number_of_files-1)
display(w)
index = 0

raw_images = []
for file in list_of_files:
    _file = os.path.join(data_folder, file)
    _image = cv2.imread(_file, cv2.IMREAD_ANYDEPTH)
    raw_images.append(_image)
    w.value = index
    index += 1

In [37]:
len(raw_images)

959

### Setting the parameters

In [39]:
sz = raw_images[0].shape
print(sz)

(246, 132)


Define the motion model

In [40]:
warp_mode = cv2.MOTION_TRANSLATION

Specify the number of iterations

In [42]:
number_of_iterations = 5000

Specify the threshold of the increment in the correlation coefficient between two iterations

In [43]:
termination_eps = 1e-10

Define termination criteria

In [44]:
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations, termination_eps)

### Looping over all the images

In [45]:
w = widgets.IntProgress()
w.max = (number_of_files-2)
display(w)
index = 0

im0 = raw_images[0]
aligned_images = []
aligned_images.append(im0)

for index in range(1, len(raw_images)):

    imx = raw_images[index]
    
    #Define 2x3 matrix and initialize the matrix identity
    warp_matrix = np.eye(2, 3, dtype=np.float32) 
    
    #Run the ECC algorithm. Result is stored in warp_matrix
    (cc, warp_matrix) = cv2.findTransformECC(im0, imx, warp_matrix, warp_mode, criteria)
    
    # use warpAffine for translation, Euclidian and Affine
    imx_aligned = cv2.warpAffine(imx, warp_matrix, (sz[1], sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    aligned_images.append(imx_aligned)
    
    w.value = index

### Export aligned images

In [48]:
from PIL import Image

In [46]:
%gui qt
output_folder = gui_fname()
%matplotlib inline

In [49]:
w2 = widgets.IntProgress()
w2.max = (number_of_files-2)
display(w2)
index = 0

for index in range(0, number_of_files):
    output_filename = os.path.join(output_folder, list_of_files[index])
    _im = aligned_images[index]
    _new_im = Image.fromarray(np.float32(_im))
    _new_im.save(output_filename)
    w2.value = index